<a href="https://colab.research.google.com/github/majd-adawieh/reinforcement-learning/blob/main/dueling-dqn/dueling-dqn_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y xvfb
!pip install pygame pyvirtualdisplay
!pip install git+https://github.com/GrupoTuring/PyGame-Learning-Environment
!pip install git+https://github.com/lusob/gym-ple

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  xvfb
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 785 kB of archives.
After this operation, 2,271 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 xvfb amd64 2:1.19.6-1ubuntu4.11 [785 kB]
Fetched 785 kB in 2s (466 kB/s)
Selecting previously unselected package xvfb.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../xvfb_2%3a1.19.6-1ubuntu4.11_amd64.deb ...
Unpacking xvfb (2:1.19.6-1ubuntu4.11) ...
Setting up xvfb (2:1.19.6-1ubuntu4.11) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [1]:
from pyvirtualdisplay import Display
Display(visible=False, size=(1400, 900)).start()

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [9]:
import gym
env = gym.make("CarRacing-v0")
env.reset()
env.render()


Track generation: 1168..1470 -> 302-tiles track


Exception ignored in: <function Viewer.__del__ at 0x0000021C240A77F0>
Traceback (most recent call last):
  File "C:\Users\majda\anaconda3\envs\torch_env\lib\site-packages\gym\envs\classic_control\rendering.py", line 185, in __del__
    self.close()
  File "C:\Users\majda\anaconda3\envs\torch_env\lib\site-packages\gym\envs\classic_control\rendering.py", line 101, in close
    self.window.close()
  File "C:\Users\majda\anaconda3\envs\torch_env\lib\site-packages\pyglet\window\win32\__init__.py", line 319, in close
    super(Win32Window, self).close()
  File "C:\Users\majda\anaconda3\envs\torch_env\lib\site-packages\pyglet\window\__init__.py", line 838, in close
    app.windows.remove(self)
  File "C:\Users\majda\anaconda3\envs\torch_env\lib\_weakrefset.py", line 114, in remove
    self.data.remove(ref(item))
KeyError: <weakref at 0x0000021C22A5CDB0; to 'Win32Window' at 0x0000021C22D38910>


True

In [2]:
import copy
import torch
import random
import gym
import gym_ple
import matplotlib

from tqdm import tqdm

import numpy as np
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib.animation as animation

from collections import deque, namedtuple
from IPython.display import HTML
from base64 import b64encode

from torch import nn
from torch.utils.data import DataLoader
from torch.utils.data.dataset import IterableDataset
from torch.optim import AdamW

from gym.wrappers import TransformObservation

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
num_gpus = torch.cuda.device_count()
print(f"device {device}, num_gpus: {num_gpus}")

C:\Users\majda\anaconda3\envs\torch_env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'gym_ple'

In [4]:
# Copied from: https://colab.research.google.com/github/deepmind/dm_control/blob/master/tutorial.ipynb#scrollTo=gKc1FNhKiVJX

def display_video(frames, framerate=30):
  height, width, _ = frames[0].shape
  dpi = 70
  orig_backend = matplotlib.get_backend()
  matplotlib.use('Agg')
  fig, ax = plt.subplots(1, 1, figsize=(width / dpi, height / dpi), dpi=dpi)
  matplotlib.use(orig_backend)
  ax.set_axis_off()
  ax.set_aspect('equal')
  ax.set_position([0, 0, 1, 1])
  im = ax.imshow(frames[0])
  def update(frame):
    im.set_data(frame)
    return [im]
  interval = 1000/framerate
  anim = animation.FuncAnimation(fig=fig, func=update, frames=frames,
                                  interval=interval, blit=True, repeat=False)
  return HTML(anim.to_html5_video())

In [5]:
class DQN(nn.Module):
  def __init__(self, hidden_size, obs_size, n_actions):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(obs_size, hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size, hidden_size),
        nn.ReLU()    
    )

    self.fc_adv = nn.Linear(hidden_size, n_actions) 
    self.fc_value = nn.Linear(hidden_size, 1)
  
  def forward(self, x):
    x = self.net(x)
    adv = self.fc_adv(x)
    value = self.fc_value(x)
    return value + adv - torch.mean(adv, dim=1, keepdim=True)

In [6]:
def policy(state, epsilon=0.):
    if torch.rand(1) < epsilon:
        return torch.randint(num_actions, (1, 1))
    else:
        av = q_network(state).detach()
        return torch.argmax(av, dim=-1, keepdim=True)

In [7]:
class ReplayMemory:
    
    def __init__(self, capacity=100000):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def insert(self, transition):
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = transition
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        assert self.can_sample(batch_size)

        batch = random.sample(self.memory, batch_size)
        batch = zip(*batch)
        return [torch.cat(items) for items in batch]

    def can_sample(self, batch_size):
        return len(self.memory) >= batch_size * 10

    def __len__(self):
        return len(self.memory)

In [8]:
class RLDataset(IterableDataset):

  def __init__(self, buffer, sample_size=400):
    self.buffer = buffer
    self.sample_size = sample_size
  
  def __iter__(self):
    for experience in self.buffer.sample(self.sample_size):
      yield experience

In [9]:
class RunningMeanStd:
    # https://en.wikipedia.org/wiki/Algorithms_for_calculating_variance#Parallel_algorithm
    def __init__(self, epsilon=1e-4, shape=()):
        self.mean = np.zeros(shape, "float64")
        self.var = np.ones(shape, "float64")
        self.count = epsilon

    def update(self, x):
        batch_mean = np.mean(x, axis=0)
        batch_var = np.var(x, axis=0)
        batch_count = x.shape[0]
        self.update_from_moments(batch_mean, batch_var, batch_count)

    def update_from_moments(self, batch_mean, batch_var, batch_count):
        self.mean, self.var, self.count = update_mean_var_count_from_moments(
            self.mean, self.var, self.count, batch_mean, batch_var, batch_count
        )


def update_mean_var_count_from_moments(
    mean, var, count, batch_mean, batch_var, batch_count
):
    delta = batch_mean - mean
    tot_count = count + batch_count

    new_mean = mean + delta * batch_count / tot_count
    m_a = var * count
    m_b = batch_var * batch_count
    M2 = m_a + m_b + np.square(delta) * count * batch_count / tot_count
    new_var = M2 / tot_count
    new_count = tot_count

    return new_mean, new_var, new_count


class NormalizeObservation(gym.core.Wrapper):
    def __init__(
        self,
        env,
        epsilon=1e-8,
    ):
        super().__init__(env)
        self.num_envs = getattr(env, "num_envs", 1)
        self.is_vector_env = getattr(env, "is_vector_env", False)
        if self.is_vector_env:
            self.obs_rms = RunningMeanStd(shape=self.single_observation_space.shape)
        else:
            self.obs_rms = RunningMeanStd(shape=self.observation_space.shape)
        self.epsilon = epsilon

    def step(self, action):
        obs, rews, dones, infos = self.env.step(action)
        if self.is_vector_env:
            obs = self.normalize(obs)
        else:
            obs = self.normalize(np.array([obs]))[0]
        return obs, rews, dones, infos

    def reset(self, **kwargs):
        return_info = kwargs.get("return_info", False)
        if return_info:
            obs, info = self.env.reset(**kwargs)
        else:
            obs = self.env.reset(**kwargs)
        if self.is_vector_env:
            obs = self.normalize(obs)
        else:
            obs = self.normalize(np.array([obs]))[0]
        if not return_info:
            return obs
        else:
            return obs, info

    def normalize(self, obs):
        self.obs_rms.update(obs)
        return (obs - self.obs_rms.mean) / np.sqrt(self.obs_rms.var + self.epsilon)


class NormalizeReward(gym.core.Wrapper):
    def __init__(
        self,
        env,
        gamma=0.99,
        epsilon=1e-8,
    ):
        super().__init__(env)
        self.num_envs = getattr(env, "num_envs", 1)
        self.is_vector_env = getattr(env, "is_vector_env", False)
        self.return_rms = RunningMeanStd(shape=())
        self.returns = np.zeros(self.num_envs)
        self.gamma = gamma
        self.epsilon = epsilon

    def step(self, action):
        obs, rews, dones, infos = self.env.step(action)
        if not self.is_vector_env:
            rews = np.array([rews])
        self.returns = self.returns * self.gamma + rews
        rews = self.normalize(rews)
        self.returns[dones] = 0.0
        if not self.is_vector_env:
            rews = rews[0]
        return obs, rews, dones, infos

    def normalize(self, rews):
        self.return_rms.update(self.returns)
        return rews / np.sqrt(self.return_rms.var + self.epsilon)

In [10]:
env = gym_ple.make("FlappyBird-v0")

/usr/local/lib/python3.7/dist-packages/gym/logger.py:30: UserWarning: WARN: Environment '<class 'gym_ple.ple_env.PLEEnv'>' has deprecated methods '_step' and '_reset' rather than 'step' and 'reset'. Compatibility code invoked. Set _gym_disable_underscore_compat = True to disable this behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [11]:
env.reset()
env.unwrapped.game_state.getGameState()

{'player_y': 256,
 'player_vel': 0,
 'next_pipe_dist_to_player': 309.0,
 'next_pipe_top_y': 53,
 'next_pipe_bottom_y': 153,
 'next_next_pipe_dist_to_player': 453.0,
 'next_next_pipe_top_y': 153,
 'next_next_pipe_bottom_y': 253}

In [12]:
list(env.unwrapped.game_state.getGameState().values())

[256, 0, 309.0, 53, 153, 453.0, 153, 253]

In [13]:
env.unwrapped.game_state.frame_skip = 4

In [14]:
class StateVectorWrapper(gym.Wrapper):

  def __init__(self, env):
    super().__init__(env)
    state = self.reset()
    self.observation_space = gym.spaces.Box(
        low=float('-inf'), 
        high=float('inf'),
        shape=state.shape
    )

  def reset(self):
    super().reset()
    state_dict = self.env.unwrapped.game_state.getGameState()
    state = list(state_dict.values())
    return np.array(state)

  def step(self, action):
    _, reward, done, info = super().step(action)
    next_state_dict = self.env.unwrapped.game_state.getGameState()
    next_state = list(next_state_dict.values())
    return np.array(next_state), reward, done, info

In [15]:
def create_environment(name):
  env = gym_ple.make(name)
  env = StateVectorWrapper(env)
  env = NormalizeObservation(env)
  env = NormalizeReward(env)
  return env

In [16]:
env = create_environment('FlappyBird-v0')
frames = []

for episode in range(10):
  done = False
  env.reset()
  while not done:
    frames.append(env.render(mode='rgb_array'))
    action = env.action_space.sample()
    _, _, done, _ = env.step(action)

In [17]:
display_video(frames)

In [18]:
env_name = "FlappyBird-v0"
env = create_environment(env_name)
obs_size = env.observation_space.shape[0]
num_actions = env.action_space.n
hidden_size = 128

q_network = DQN(128, obs_size, num_actions).to(device)
target_q_network = copy.deepcopy(q_network).eval().to(device)

In [24]:
def dueling_deep_q_learning(q_network, policy, episodes, 
                    alpha=0.0001, batch_size=32, gamma=0.99, epsilon=0.2):
    
    optim = AdamW(q_network.parameters(), lr=alpha)
    memory = ReplayMemory()
    stats = {'MSE Loss': [], 'Returns': []}
    for episode in tqdm(range(1, episodes + 1)):
        state = env.reset()
        state = torch.from_numpy(state).unsqueeze(dim=0)
        done = False
        ep_return = 0
        while not done:
            state = state.float().to(device)
            action = policy(state, epsilon)
            next_state, reward, done, _ = env.step(action)
            
            next_state = torch.from_numpy(next_state).unsqueeze(dim=0)
            reward = torch.tensor(reward).view(1, -1)
            done = torch.tensor(done).view(1, -1)
            state = state.cpu()
            action = action.cpu()
            memory.insert([state, action, reward, done, next_state])
           
            if memory.can_sample(batch_size):
                state_b, action_b, reward_b, done_b, next_state_b = memory.sample(batch_size)
                state_b = state_b.to(device)
                reward_b = reward_b.to(device)
                action_b = action_b.to(device)
                done_b = done_b.to(device)
                next_state_b = next_state_b.float().to(device)
                
                qsa_b = q_network(state_b).gather(1, action_b)
                next_qsa_b = target_q_network(next_state_b)
                next_qsa_b = torch.max(next_qsa_b, dim=-1, keepdim=True)[0]
                next_qsa_b.float().to(device)
                target_b = reward_b + ~done_b * gamma * next_qsa_b
                loss = F.mse_loss(qsa_b.float(), target_b.float())
                q_network.zero_grad()
                loss.backward()
                optim.step()
                stats['MSE Loss'].append(loss)  
                
            state = next_state
            ep_return += reward.item()
            
        
        stats['Returns'].append(ep_return)
        
        if episode % 10 == 0:
            target_q_network.load_state_dict(q_network.state_dict())

    return stats

In [25]:
stats = dueling_deep_q_learning(q_network,policy,3000)

 81%|████████  | 2429/3000 [1:01:23<14:25,  1.52s/it]


KeyboardInterrupt: ignored

In [21]:
frames = []

for episode in range(2):
  done = False
  state = env.reset()
  while not done:
    frames.append(env.render(mode='rgb_array'))
    state = torch.from_numpy(state).unsqueeze(dim=0).float().to(device)
    action = policy(state)
    state, _, done, _ = env.step(action)

In [22]:
display_video(frames)